# AI based Natural disaster analysis

In [1]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


In [2]:
tensorflow.__version__

'2.7.2'

In [3]:
tensorflow.keras.__version__

'2.7.0'

In [4]:
pwd

'/home/wsuser/work'

In [5]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_4628cf0ea7244bccb7c2d765a81ac335 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='rBHiwP0FpAIWsHNMSbxhhZC2ly3qSPo2EvLVJgUQid-G',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_4628cf0ea7244bccb7c2d765a81ac335.get_object(Bucket='naturaldisaster-donotdelete-pr-x8qdlmowdjen6y', Key='dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [7]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [8]:
import os
filenames=os.listdir('/home/wsuser/work/dataset')

In [9]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'/home/wsuser/work/dataset/train_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'/home/wsuser/work/dataset/test_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical') 

Found 742 images belonging to 4 classes.
Found 198 images belonging to 4 classes.


In [10]:
print(x_train.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [11]:
print(x_test.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [12]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 220, 1: 156, 2: 198, 3: 168})

In [13]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and poolingo
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=4, activation='softmax')) # softmax for more than 2



In [14]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 32)        9

In [15]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

In [16]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

/tmp/wsuser/ipykernel_164/549542485.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/20
149/149 [==============================] - 35s 230ms/step - loss: 1.2540 - accuracy: 0.4218 - val_loss: 1.1828 - val_accuracy: 0.5000
Epoch 2/20
149/149 [==============================] - 34s 229ms/step - loss: 1.0294 - accuracy: 0.5472 - val_loss: 1.2592 - val_accuracy: 0.3636
Epoch 3/20
149/149 [==============================] - 34s 226ms/step - loss: 0.9490 - accuracy: 0.6132 - val_loss: 0.9212 - val_accuracy: 0.6667
Epoch 4/20
149/149 [==============================] - 33s 225ms/step - loss: 0.8681 - accuracy: 0.6402 - val_loss: 0.8499 - val_accuracy: 0.7020
Epoch 5/20
149/149 [==============================] - 34s 229ms/step - loss: 0.7737 - accuracy: 0.6914 - val_loss: 0.7643 - val_accuracy: 0.6616
Epoch 6/20
149/149 [==============================] - 34s 226ms/step - loss: 0.7165 - accuracy: 0.7102 - val_loss: 0.8288 - val_accuracy: 0.6919
Epoch 7/20
149/149 [==============================] - 33s 222ms/step - loss: 0.6730 - accuracy: 0.7318 - val_loss: 0.9417 - val_ac

In [17]:
# Save the model
classifier.save('disaster.h5')

In [18]:
!tar zcvf natural-disaster-new.tgz disaster.h5

disaster.h5


In [19]:
ls -1

dataset/
disaster.h5
natural-disaster-new.tgz


In [20]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 20.0 MB/s eta 0:00:01


In [21]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

In [22]:
 from ibm_watson_machine_learning import APIClient
wml_credentials={
        "url":"https://us-south.ml.cloud.ibm.com",
        "apikey":"tLvqhgailCsiFTL8LX5pDMTvUdAvXCiEyMFCLaTT8_mv"
}
client=APIClient(wml_credentials)

In [23]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)["metadata"]['id'])

In [24]:
space_uid=guid_from_space_name(client,'model')
print("Space UID = "+space_uid)

Space UID = c87cddad-9c3a-47f6-9f1c-e7027e007fe3


In [25]:
client.set.default_space(space_uid)

'SUCCESS'

In [26]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [27]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [28]:
model_details = client.repository.store_model(model='natural-disaster-new.tgz', meta_props= {
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [29]:
model_id

'23d62951-0ab6-4879-a62e-4cfff38ebed9'

In [30]:
client.repository.download(model_id,'my-model.tar.gz')

Successfully saved model content to file: 'my-model.tar.gz'


'/home/wsuser/work/my-model.tar.gz'

### Predicting our results

In [41]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
#model = load_model("disaster.h5") #loading the model for testing